In [17]:
#importing necessary libraries and packages

import pandas as pd
import geopandas as gpd

In [18]:
#reading in data
#35 rows, one for each beat, with percents of [race] by beat
by_beat = pd.read_csv('/Users/morganbernstein/Desktop/Data+/SToPAGetDurhamData/10 Clean Data/GROUPEDBY_BEAT.csv', index_col = [0])

In [19]:
#making columns to show the percent of arrests per race per police beat
by_beat["percent_white_A"] = round(by_beat["White"] / by_beat["total"], 5)
by_beat["percent_black_A"] = round(by_beat["Black"] / by_beat["total"], 5)
by_beat["percent_poc_A"] = round(1 - by_beat["percent_white_A"], 5)

by_beat.head()

,LAWBEAT,OBJECTID,PERIMETER,ShapeSTAre,ShapeSTLen,index_right,arrestnumber,age,casenumber,year,distance,Black,White,Asian,Unknown,Indigenous,total,percent_white_A,percent_black_A,percent_poc_A
0,111,25300,1.447605e+07,1.017188e+10,1.447605e+07,6932453,161251565,17640,10100612646,1022141,0.000000,437,64,5,0,0,506,0.12648,0.86364,0.87352
1,112,35292,1.817597e+07,1.439314e+10,1.817597e+07,8205390,220195843,24389,13759026946,1397815,0.000000,562,128,1,1,0,692,0.18497,0.81214,0.81503
2,113,40548,1.351657e+07,1.412114e+10,1.351657e+07,6790155,208674527,22175,13113632965,1321149,0.000000,539,106,6,2,1,654,0.16208,0.82416,0.83792
3,114,37095,3.009554e+07,3.442838e+10,3.009554e+07,3051224,181450891,19209,11401157693,1149433,0.002620,478,90,1,0,0,569,0.15817,0.84007,0.84183
4,121,21572,1.514383e+07,1.172609e+10,1.514383e+07,7290514,169441503,20120,10730768062,1068758,0.012345,352,170,3,3,1,529,0.32136,0.66541,0.67864


In [20]:
#reading in file of demographic data and geometry data
demo = gpd.read_file("/Users/morganbernstein/Downloads/demo_beat_data/demographic_summary_arrests_per_beat.shp")


#merging dataset with the total arrests by beat
full = pd.merge(demo, by_beat[["LAWBEAT", "Black", "White", "Asian", "Unknown", "Indigenous", "total"]], left_on = "Beat", right_on = "LAWBEAT")
full.set_index("Beat", inplace = True)


In [21]:
#making new columns to represent the percent of residents and percent of arrests per beat

full["percent_white"] = round(full["white_only"] / full["Total_"], 5)
full["percent_poc"] = round(full["poc"] / full["Total_"], 5)
full["percent_apb"] = round(full["any_part_b"] / full["Total_"], 5)

full["percent_white_A"] = round(full["White"] / full["total"], 5)
full["percent_black_A"] = round(full["Black"] / full["total"], 5)
full["percent_poc_A"] = round(1 - full["percent_white_A"], 5)

#making a log ratio column to show the percent of arrests / percent of residents
# per race for each of the police beats
full["log_B"] = full["percent_black_A"] / full["percent_apb"]
full["log_W"] = round(full["percent_white_A"] / full["percent_white"], 5)
full["log_POC"] = round(full["percent_poc_A"] / full["percent_poc"], 5)

In [22]:
#dropping duplicates
full = full.drop_duplicates(subset = "percent_black_A")
#re-ordering columns within the dataset
full = full[["percent_black_A", "percent_poc_A", "percent_white_A", "percent_white", "percent_apb", "percent_poc"]]
full = full[["percent_apb", "percent_black_A", "percent_white", "percent_white_A", "percent_poc", "percent_poc_A"]]
#renaming coumns within the dataset
full = full.rename(columns={'percent_apb': 'Percent Black Residents', 'percent_black_A': 'Percent Black Arrests'})
full = full.rename(columns={'percent_white': 'Percent White Residents', 'percent_white_A': 'Percent White Arrests'})
full = full.rename(columns={'percent_poc': 'Percent POC Residents', 'percent_poc_A': 'Percent POC Arrests'})
full

,Percent Black Residents,Percent Black Arrests,Percent White Residents,Percent White Arrests,Percent POC Residents,Percent POC Arrests
Beat,,,,,,
111,0.36032,0.86364,0.43412,0.12648,0.57433,0.87352
112,0.38377,0.81214,0.40181,0.18497,0.60586,0.81503
113,0.35932,0.82416,0.43211,0.16208,0.57939,0.83792
114,0.36920,0.84007,0.42702,0.15817,0.57946,0.84183
121,0.37472,0.66541,0.42215,0.32136,0.58894,0.67864
122,0.36749,0.74635,0.41867,0.25365,0.59274,0.74635
123,0.36951,0.78795,0.43291,0.19420,0.57362,0.80580
124,0.36562,0.48276,0.43866,0.51724,0.57411,0.48276
211,0.37520,0.51621,0.42120,0.47382,0.59227,0.52618


In [23]:
#looking for rows where the percent of white arrests is 
# greater than the percent of black arrests
full.loc[full["Percent White Arrests"] > full["Percent POC Arrests"]]

,Percent Black Residents,Percent Black Arrests,Percent White Residents,Percent White Arrests,Percent POC Residents,Percent POC Arrests
Beat,,,,,,
124,0.36562,0.48276,0.43866,0.51724,0.57411,0.48276


In [24]:
#looking for rows where the percent of white residents is 
# greater than the percent of black residents
full.loc[full["Percent White Residents"] < full["Percent Black Residents"]]

,Percent Black Residents,Percent Black Arrests,Percent White Residents,Percent White Arrests,Percent POC Residents,Percent POC Arrests
Beat,,,,,,


In [25]:
#printing out columns in the table to use for a visual
full[["Percent Black Residents", "Percent Black Arrests", "Percent White Residents", "Percent White Arrests"]].mean()

Percent Black Residents    0.372623
Percent Black Arrests      0.722512
Percent White Residents    0.423206
Percent White Arrests      0.268514
dtype: float64

In [30]:
#looking into mins and maxs of columns as well as ranges
full[["Percent White Arrests"]].max() 
#full[["Percent White Arrests"]].min()

Percent White Arrests    0.51724
dtype: float64

In [31]:
#looking at standard deviations of columns
full.std()

Percent Black Residents    0.008638
Percent Black Arrests      0.108393
Percent White Residents    0.010264
Percent White Arrests      0.106088
Percent POC Residents      0.011098
Percent POC Arrests        0.106088
dtype: float64